<a href="https://colab.research.google.com/github/JeffHCross/PyDirTreemap/blob/main/PyDirTreeMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from pathlib import Path
def gen_file_row(f):
  return {'parent':str(f.parent),'type':f.suffix,'size':f.stat().st_size}

def process_dir(d):
  paths = {}
  dir_size = 0
  try:
    for p in d.iterdir():
      try:
        if p.is_dir():
          res = process_dir(p)
          paths.update(res)
          dir_size += res[str(p)]['size']
        elif p.is_file():
          res = gen_file_row(p)
          paths[str(p)] = res
          dir_size += res['size']
      except (FileNotFoundError, PermissionError):
        continue
  except (FileNotFoundError, PermissionError):
    return paths
  paths[str(d)]={'parent':str(d.parent),'type':'directory','size':dir_size}
  return paths

In [46]:
import pandas as pd
data=process_dir(Path('/tensorflow-1.15.2'))
df = pd.DataFrame.from_dict(data, orient='index').reset_index().rename(columns={'index':'path'})
with pd.option_context('display.max_colwidth', None):
 display(df.loc[df['type'] != 'directory'].nlargest(10,'size'))

,path,parent,type,size
4373,/tensorflow-1.15.2/python3.7/tensorflow_core/python/_pywrap_tensorflow_internal.so,/tensorflow-1.15.2/python3.7/tensorflow_core/python,.so,1001409816
1569,/tensorflow-1.15.2/python3.7/tensorflow_core/libtensorflow_framework.so.1,/tensorflow-1.15.2/python3.7/tensorflow_core,.1,36157560
10281,/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/fused_conv/python/ops/_fused_conv2d_bias_activation_op.so,/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/fused_conv/python/ops,.so,17834576
8586,/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/bigtable/python/ops/_bigtable.so,/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/bigtable/python/ops,.so,16647056
10101,/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/kinesis/_dataset_ops.so,/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/kinesis,.so,10332280
42,/tensorflow-1.15.2/python3.7/mpi4py/MPI.cpython-37m-x86_64-linux-gnu.so,/tensorflow-1.15.2/python3.7/mpi4py,.so,7001256
14312,/tensorflow-1.15.2/python3.7/pretty_midi/TimGM6mb.sf2,/tensorflow-1.15.2/python3.7/pretty_midi,.sf2,5994284
9199,/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/tensor_forest/libforestprotos.so,/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/tensor_forest,.so,4735504
8825,/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/reduce_slice_ops/python/ops/_reduce_slice_ops.so,/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/reduce_slice_ops/python/ops,.so,4503280
10152,/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/image/python/ops/_single_image_random_dot_stereograms.so,/tensorflow-1.15.2/python3.7/tensorflow_core/contrib/image/python/ops,.so,4486640


In [62]:
import plotly.express as px
fig = px.treemap(df.nlargest(50,'size'),
    branchvalues = "total",
    names = 'path',
    parents = 'parent',
    values = 'size',
    title = '50 largest directories/files'
)
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()